In [2]:
import pathlib
import pickle as pck

%cd -q "/home/ebertp/work/code/cubi/project-run-hgsvc-hybrid-assemblies/notebooks"
_PROJECT_CONFIG_NB = str(pathlib.Path("00_project_config.ipynb").resolve(strict=True))

%run $_PROJECT_CONFIG_NB

assembler = "verkko"

cache_file_chrom_t2t_status = PROJECT_NB_CACHE.joinpath(f"chrom_t2t_status.{assembler}.pck")

with open(cache_file_chrom_t2t_status, "rb") as cache:
    data = pck.load(cache)

out_table = []

t2t_label_remap = {
    "other": "not T2T",
    "t2t_ctg": "T2T contig",
    "t2t_scf": "T2T scaffold"
}

for sample_key, infos in data.items():
    if len(sample_key) == 2:
        # just maps sample - sex
        continue
    sample, hap, chrom = sample_key
    sex, contig_id, t2t_label = infos[:3]
    remapped_label = t2t_label_remap[t2t_label]
    row = (sample, sex, hap, chrom, remapped_label, contig_id)
    out_table.append(row)

df = pd.DataFrame.from_records(
    out_table,
    columns=["sample", "sex", "haplotype", "chromosome", "T2T label", "contig ID"]
)

def get_chrom_order_number(chrom_name):
    try:
        num = int(chrom_name.strip("chr"))
    except ValueError:
        if chrom_name == "chrX":
            num = 23
        elif chrom_name == "chrY":
            num = 24
        else:
            raise
    return num

df["chrom_number"] = df["chromosome"].apply(get_chrom_order_number)
df.sort_values(["sample", "haplotype", "chrom_number"], inplace=True)
df.drop("chrom_number", axis=1, inplace=True)

table_sx_vrk_t2t = TABLE_OUT_SUPPL.joinpath(
    f"table_SXPE_{assembler}_t2t_chrom.tsv"
)

df.to_csv(table_sx_vrk_t2t, sep="\t", header=True, index=False)